In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import itertools
import json
from pathlib import Path
from datetime import timedelta, datetime
from scipy.sparse import csr_matrix, vstack


import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [ ]:
# Path("/media/hpi_share/")#Path("//FS23/projekte$/MP2021/MPWS2021/MPWS2021FN1")
mp_drive_dir = Path("../../../")
mp_plot_dir = mp_drive_dir / "plots"
input_path = Path("../../matched-infoboxes-extracted/")
input_data = list(input_path.rglob("*.json"))
files = [x for x in input_data if x.is_file()]
len(files)  # total 580


In [ ]:
num_iteratons = 1
number_of_files = 5
number_of_files_start = 5
num_edits = 0
num_change_tuples = 0
typo_lst = []
for _ in range(num_iteratons):
    revision_tuples = []
    change_tuples = []
    for file in tqdm(files[number_of_files_start:number_of_files_start+number_of_files]):
        with open(file, 'r', encoding='utf-8') as f:
            for jsonObj in f:
                single_edit = json.loads(jsonObj)
                num_edits += 1
                title = single_edit['pageTitle']
                pageID = single_edit['pageID']
                key = single_edit['key']
                template = single_edit['template'] if 'template' in single_edit.keys(
                ) else None
                changes = single_edit['changes']
                validFrom = single_edit['validFrom']
                revisionId = single_edit['revisionId']
                changeType = single_edit['type']
                attributes = single_edit['attributes'] if 'attributes' in single_edit.keys(
                ) else None
                user_name = single_edit['user']['username'] if 'username' in single_edit['user'].keys(
                ) else None
                user_id = single_edit['user']['id'] if 'id' in single_edit['user'].keys(
                ) else None
                user_ip = single_edit['user']['ip'] if 'ip' in single_edit['user'].keys(
                ) else None
                revision_tuples.append(
                    (title, pageID, key, template, name, revisionId, changeType, user_name, user_id, user_ip, attributes))

                for change in changes:
                    num_change_tuples += 1
                    name = change['property']['name']
                    current_value = change['currentValue'] if 'currentValue' in change.keys(
                    ) else None
                    previous_value = change['previousValue'] if 'previousValue' in change.keys(
                    ) else None
                    validTo = change['valueValidTo'] if 'valueValidTo' in change.keys(
                    ) else None
                    change_tuples.append((title, pageID, key, template, name, previous_value,
                                          current_value, validFrom, validTo, revisionId, changeType, user_name, user_id, user_ip, attributes))
    number_of_files_start += number_of_files


In [ ]:
# data = pd.DataFrame(change_tuples, columns=['key', 'pageID', 'title', 'user','name', 'previous_value', 'current_value', 'timestamp', 'edit_type'])
# data['timestamp'] = pd.to_datetime(data['timestamp']).dt.tz_localize(None)
data = pd.DataFrame(change_tuples, columns=['pageTitle', 'pageID', 'key', 'template', 'name', 'previous_value',
                    'current_value', 'validFrom', 'validTo', 'revisionId', 'change_type', 'user_name', 'user_id', 'user_ip', 'attributes'])
data['validFrom'] = pd.to_datetime(data['validFrom'])
data['validTo'] = pd.to_datetime(data['validTo'])


In [ ]:
data.head(3)

In [ ]:
print("Unique user ids:", data['user_id'].nunique())
print("Unique user ips:", data['user_ip'].nunique())
print("Unique users (unique ip/id combinations):", len(data[['user_id', 'user_ip']].drop_duplicates())) #including both NaN/None
#data['user_id'].nunique()+data['user_ip'].nunique()

In [ ]:
# user ids and ips are none?
data.query('user_id.isnull() and user_ip.isnull()').head(3)

In [ ]:
# Mostly Deletes and Update have no user
data.query('user_id.isnull() and user_ip.isnull()').groupby('change_type').count()

In [ ]:
# sanity check if one revid has one user
data[["revisionId","user_id","user_ip"]].groupby("revisionId").nunique().head(3)

In [ ]:
data[["revisionId","user_id","user_ip"]].groupby(["user_id","user_ip"])

In [ ]:
data_revs = pd.DataFrame(revision_tuples, columns=['pageTitle', 'pageID', 'key', 'template', 'name', 'revisionId', 'change_type', 'user_name', 'user_id', 'user_ip', 'attributes'])
data_revs.head(3)

In [ ]:
print("max creations/updates/deletions of a single user")
print("Create",data_revs.query('change_type=="CREATE"').groupby("user_id").count()['revisionId'].max())
print("Delete",data_revs.query('change_type=="DELETE"').groupby("user_id").count()['revisionId'].max())
print("Update",data_revs.query('change_type=="UPDATE"').groupby("user_id").count()['revisionId'].max())

- How many different pages a user works on
- how many different users work on a single page
- at which time a user makes changes
- does a user make changes in a specific order multiple times